| Equation | Form     | Type       | Describes                |
|----------|---------|------------|--------------------------|
| Laplace  | ∇²u = 0 | Elliptic   | Steady-state fields      |
| Heat     | ∂u/∂t = α ∇²u | Parabolic | Diffusion / conduction   |
| Wave     | ∂²u/∂t² = c² ∇²u | Hyperbolic | Wave propagation         |


In [ ]:
from manim import *
import numpy as np

class PDEComparison(ThreeDScene):
    def construct(self):

        # Intro title (static, HUD style)
        title = Text("Partial Differential Equations", font_size=42, color=YELLOW)
        self.add_fixed_in_frame_mobjects(title)
        title.to_edge(UP)
        self.play(FadeIn(title))
        self.wait(2)

        # Set camera
        self.set_camera_orientation(phi=65*DEGREES, theta=45*DEGREES)

        # ======================================================
        # 1. LAPLACE EQUATION
        # ======================================================
        laplace_label = Text("Laplace Equation (∇²u = 0)", font_size=32, color=BLUE)
        self.add_fixed_in_frame_mobjects(laplace_label)
        laplace_label.to_edge(DOWN)

        def laplace_func(u, v):
            return np.sin(u) * np.cosh(v) / np.cosh(2)

        axes = ThreeDAxes(x_range=[-2,2], y_range=[-2,2], z_range=[-2,2])
        laplace_surface = Surface(
            lambda u, v: np.array([u, v, laplace_func(u, v)]),
            u_range=[-2, 2], v_range=[-2, 2],
            resolution=(30, 30),
            fill_opacity=0.7,
            checkerboard_colors=[BLUE_E, BLUE_A],
        )

        # Vector field = gradient of potential
        def laplace_grad(u, v):
            du = np.cos(u) * np.cosh(v)/np.cosh(2)
            dv = np.sin(u) * np.sinh(v)/np.cosh(2)
            return np.array([du, dv, 0])

        vectors = VGroup()
        for x in np.linspace(-2,2,6):
            for y in np.linspace(-2,2,6):
                grad = laplace_grad(x,y)
                vec = Arrow3D(
                    start=np.array([x,y,0]),
                    end=np.array([x,y,0]) + 0.5*grad/np.linalg.norm(grad+1e-6),
                    color=BLUE
                )
                vectors.add(vec)

        self.play(Create(axes), FadeIn(laplace_surface), FadeIn(vectors), FadeIn(laplace_label))
        self.begin_3dillusion_camera_rotation(rate=0.1)
        self.wait(4)
        self.stop_3dillusion_camera_rotation()
        self.play(FadeOut(laplace_surface), FadeOut(vectors), FadeOut(axes), FadeOut(laplace_label))

        # ======================================================
        # 2. HEAT EQUATION
        # ======================================================
        heat_label = Text("Heat Equation (∂u/∂t = α∇²u)", font_size=32, color=RED)
        self.add_fixed_in_frame_mobjects(heat_label)
        heat_label.to_edge(DOWN)

        def heat_func(u, v, t):
            return np.exp(-t) * np.exp(-(u**2+v**2))

        t_tracker = ValueTracker(0)

        heat_surface = always_redraw(
            lambda: Surface(
                lambda u,v: np.array([u,v,heat_func(u,v,t_tracker.get_value())]),
                u_range=[-2,2], v_range=[-2,2],
                resolution=(30,30),
                fill_opacity=0.7,
                checkerboard_colors=[RED_E, RED_A],
            )
        )

        # Radial vector field showing diffusion
        vectors = VGroup()
        for x in np.linspace(-2,2,6):
            for y in np.linspace(-2,2,6):
                vec = Arrow3D(
                    start=np.array([x,y,0]),
                    end=np.array([x,y,0]) + 0.4*np.array([x,y,0])/np.linalg.norm([x,y])+1e-6,
                    color=RED
                )
                vectors.add(vec)

        self.play(FadeIn(heat_surface), FadeIn(vectors), FadeIn(heat_label))
        self.play(t_tracker.animate.set_value(5), run_time=6, rate_func=linear)
        self.play(FadeOut(heat_surface), FadeOut(vectors), FadeOut(heat_label))

        # ======================================================
        # 3. WAVE EQUATION
        # ======================================================
        wave_label = Text("Wave Equation (∂²u/∂t² = c²∇²u)", font_size=32, color=GREEN)
        self.add_fixed_in_frame_mobjects(wave_label)
        wave_label.to_edge(DOWN)

        def wave_func(u, v, t):
            return np.sin(2*np.sqrt(u**2+v**2)-2*t)/(1+u**2+v**2)

        t_wave = ValueTracker(0)

        wave_surface = always_redraw(
            lambda: Surface(
                lambda u,v: np.array([u,v,wave_func(u,v,t_wave.get_value())]),
                u_range=[-3,3], v_range=[-3,3],
                resolution=(40,40),
                fill_opacity=0.7,
                checkerboard_colors=[GREEN_E, GREEN_A],
            )
        )

        # Oscillatory vertical arrows
        vectors = always_redraw(lambda: VGroup(
            *[Arrow3D(
                start=np.array([x,y,0]),
                end=np.array([x,y,0]) + np.array([0,0,0.3*np.sin(t_wave.get_value()+x+y)]),
                color=GREEN
            )
              for x in np.linspace(-3,3,6)
              for y in np.linspace(-3,3,6)]
        ))

        self.play(FadeIn(wave_surface), FadeIn(vectors), FadeIn(wave_label))
        self.play(t_wave.animate.set_value(10), run_time=6, rate_func=linear)
        self.play(FadeOut(wave_surface), FadeOut(vectors), FadeOut(wave_label))

        # Outro
        outro = VGroup(
            MathTex(r"\nabla^2 u = 0", color=BLUE),
            MathTex(r"\frac{\partial u}{\partial t} = \alpha \nabla^2 u", color=RED),
            MathTex(r"\frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u", color=GREEN),
        ).arrange(DOWN, buff=1)

        self.add_fixed_in_frame_mobjects(outro)
        self.play(FadeIn(outro))
        self.wait(3)
        self.play(FadeOut(outro), FadeOut(title))


%manim -qk -v error PDEComparison